In [1]:
from __future__ import print_function 
import os, sys, h5py
import numpy as np
from six.moves import cPickle
from collections import OrderedDict

import tensorflow as tf
sys.path.append('..')
from tfomics import neuralnetwork_dev as nn
from tfomics import neuralbuild as nb
from tfomics import utils, learn

# import models
from model_zoo import test_model

In [2]:
filename = 'processed_dataset.hdf5'
#data_path = 'D:/Dropbox/TFconnect'
data_path = '/home/peter/Code/tensorflow/data'
filepath = os.path.join(data_path,filename)

group_name = ['processed_data']
dataset = h5py.File(filepath,'r')
%time dtf = np.array(dataset['/'+group_name[0]+'/dtf'])
ltf = np.array(dataset['/'+group_name[0]+'/ltf'])
dtf_crossval = np.array(dataset['/'+group_name[0]+'/dtf_crossval'])
ltf_crossval = np.array(dataset['/'+group_name[0]+'/ltf_crossval'])

X_train = dtf[:,:,:,:]
y_train = ltf[:,:]
X_valid = dtf_crossval[:,:,:,:]
y_valid = ltf_crossval[:,:]

CPU times: user 444 ms, sys: 1.02 s, total: 1.46 s
Wall time: 1.46 s


In [3]:
# get shapes
num_data, height, width, dim = X_train.shape
input_shape=[None, height, width, dim]
num_labels = y_train.shape[1]  

In [4]:
model_layers, optimization = test_model.model(input_shape, num_labels)
nnbuild = nb.NeuralBuild(model_layers)
network, placeholders, hidden_feed_dict = nnbuild.get_network_build()

# build neural network class
nnmodel = nn.NeuralNet(network, placeholders, hidden_feed_dict)
nnmodel.inspect_layers()

# set output file paths

results_path = utils.make_directory(data_path, 'results')
results_path = utils.make_directory(results_path, 'tfomics')
output_name = 'test'
filepath = os.path.join(results_path, output_name)

# compile neural trainer
nntrainer = nn.NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

----------------------------------------------------------------------------
Network architecture:
----------------------------------------------------------------------------
layer1: input
(?, 3, 330, 1)
layer2: conv1
(?, 2, 326, 32)
layer3: conv1_active
(?, 2, 326, 32)
layer4: conv1_dropout
(?, 2, 326, 32)
layer5: resid1_1resid
(?, 2, 326, 32)
layer6: resid1_1resid_norm
(?, 2, 326, 32)
layer7: resid1_1resid_active
(?, 2, 326, 32)
layer8: resid1_dropout1
(?, 2, 326, 32)
layer9: resid1_2resid
(?, 2, 326, 32)
layer10: resid1_2resid_norm
(?, 2, 326, 32)
layer11: resid1_resid_sum
(?, 2, 326, 32)
layer12: resid1_resid
(?, 2, 326, 32)
layer13: resid1_batch
(?, 2, 326, 32)
layer14: resid1_dropout
(?, 2, 326, 32)
layer15: conv2
(?, 1, 322, 64)
layer16: conv2_batch
(?, 1, 322, 64)
layer17: conv2_active
(?, 1, 322, 64)
layer18: conv2_dropout
(?, 1, 322, 64)
layer19: resid2_1resid
(?, 1, 322, 64)
layer20: resid2_1resid_norm
(?, 1, 322, 64)
layer21: resid2_1resid_active
(?, 1, 322, 64)
layer22: r

In [5]:
train = {'inputs': [X_train], 'targets': y_train}
valid = {'inputs': [X_valid], 'targets': y_valid}
data = {'train': train, 'valid': valid}
learn.train_minibatch(nntrainer, data, batch_size=100, num_epochs=50, 
                    patience=10, verbose=2, shuffle=True)

Epoch 1 out of 50 
[==============================] 100.0% -- time=0s -- loss=0.75856 -- accuracy=50.04%    
  valid loss:		0.69561
  valid accuracy:	0.49846+/-0.00000
  valid auc-roc:	0.49901+/-0.00000
  valid auc-pr:		0.49908+/-0.00116
  lower cross-validation found
  saving model to:  /home/peter/Code/tensorflow/data/results/tfomics/test_best.ckpt
Epoch 2 out of 50 
[==============================] 100.0% -- time=0s -- loss=0.69684 -- accuracy=50.01%    
  valid loss:		0.69787
  valid accuracy:	0.50095+/-0.00000
  valid auc-roc:	0.50283+/-0.00000
  valid auc-pr:		0.50230+/-0.00103
Epoch 3 out of 50 
[==============================] 100.0% -- time=0s -- loss=0.69736 -- accuracy=50.02%    
  valid loss:		0.69685
  valid accuracy:	0.50082+/-0.00000
  valid auc-roc:	0.49904+/-0.00000
  valid auc-pr:		0.49942+/-0.00142
Epoch 4 out of 50 
[==============================] 100.0% -- time=0s -- loss=0.58706 -- accuracy=66.39%    
  valid loss:		0.46456
  valid accuracy:	0.81678+/-0.00000
  v

KeyboardInterrupt: 

In [ ]:

batch_mean, batch_var = tf.nn.moments(, [0])

In [ ]:
x = tf.identity([1., 2.])

In [ ]:
ema_op = ema.apply(x)

In [ ]:
sess = tf.Session()
sess.run(ema)